In [12]:
import pandas as pd
import swifter

import numpy as np 
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import community
import time

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [13]:
%%time
df1= pd.read_excel("obs_asigna_osee.xlsx")
print(df1.shape)
df1.head()

(81488, 5)
Wall time: 8.47 s


,CODIGO_MODULAR,SERIE_EQUIPO_OSEE,obs,flg_cat,flg_bin
0,200022,T0000000001,"Ninguna, esta operativa.",97,1
1,200220,T0000000002,La niña vive con su abuela Aurelia Costilla Fl...,6,1
2,200238,T0000000003,Se asignó la tableta a la estudiante pero aún ...,14,0
3,200220,T0000000004,La Madre vive en Anexo y no pudo actualizar su...,1,0
4,200220,T0000000005,La madre vive en Anexo con la Estudiante y no ...,1,0


# 1. DataCleaning

## 1.1. Clean StopWords

In [14]:
import spacy
spacy.prefer_gpu()
spacy.cli.download('es_core_news_md')
nlp = spacy.load('es_core_news_md')

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [15]:
#Añadiendo nuevos stopwords
#==========================
sw_nuevos = ['rt','tv','https','mineduperu','ja','q','xd', 'iberio','iberia','iberica','jaja','jajajaja','http', 'click', 'clase','ano','escolar']
for sw in sw_nuevos:
    nlp.Defaults.stop_words.add(sw)
    nlp.vocab[sw].is_stop = True
    
#Eliminando Sw x defecto y personalizados
#========================================

def QuitarSwTodos(tokens):
    doc = nlp(str(tokens))
    n_tokens = [token.text for token in doc if not token.is_stop]   

    #lematizando
    #==========    
    cad = ' '.join([str(item) for item in n_tokens])  
    doc = nlp(cad)
    lemas = [tok.lemma_.lower() for tok in doc]
    cad = ' '.join([str(item) for item in lemas])  
    # print('cad',cad) 
    # print('lemas',lemas) 
    return cad

In [16]:
df1 = df1.head(20000) #seleccionamos los primeros 20mil para la demo
df1['oracion_final_str'] = df1['obs'].swifter.apply(lambda x: QuitarSwTodos(x))
df = df1.copy()

Pandas Apply:   0%|          | 0/20000 [00:00<?, ?it/s]

## 1.2. Clean Words (Speller)

In [17]:
from spellchecker import SpellChecker
import re

spellsp = SpellChecker(language='es', distance=1, case_sensitive= False)

In [18]:
def clean_html(html):
        # First we remove inline JavaScript/CSS:
    cleaned = re.sub(r"(?is)<(script|style).*?>.*?(</\1>)", "", html.strip())
    # Then we remove html comments. This has to be done before removing regular
    # tags since comments can contain '>' characters.
    cleaned = re.sub(r"(?s)<!--(.*?)-->[\n]?", "", cleaned)
    # Next we can remove the remaining tags:
    cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
    # Finally, we deal with whitespace
    cleaned = re.sub(r"&nbsp;", " ", cleaned)
    cleaned = re.sub(r"[\r\n]", " ", cleaned)
    cleaned = re.sub(r"[(http(s)?):\/\/(www\.)?a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)", " ", cleaned)
    cleaned = re.sub(r"  ", " ", cleaned)
    cleaned = re.sub(r"  ", " ", cleaned)
    return cleaned.strip()
    raise NotImplementedError ("To remove HTML markup, use BeautifulSoup's get_text() function")

In [19]:
df['oracion_final_str'] = df.oracion_final_str.swifter.apply(lambda oracion: clean_html(str(oracion).lower()))
df['oracion_final_str'] = df.oracion_final_str.swifter.apply(lambda oracion: re.findall('[a-zA-Záéíóúü]+', oracion))

df['length'] = df.oracion_final_str.swifter.apply(lambda oracion: len(oracion))
df = df[df['length']>=3]
print(df.shape)
df.head(3)

Pandas Apply:   0%|          | 0/20000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20000 [00:00<?, ?it/s]

(9659, 7)


,CODIGO_MODULAR,SERIE_EQUIPO_OSEE,obs,flg_cat,flg_bin,oracion_final_str,length
1,200220,T0000000002,La niña vive con su abuela Aurelia Costilla Fl...,6,1,"[ni, a, vivir, abuela, aurelia, costilla, flor...",10
2,200238,T0000000003,Se asignó la tableta a la estudiante pero aún ...,14,0,"[asignar, tableta, estudiante, entregar, estud...",6
3,200220,T0000000004,La Madre vive en Anexo y no pudo actualizar su...,1,0,"[madre, vivir, anexo, actualizar, dni, motivo,...",7


In [20]:
# Corrigiendo palabras mal escritas (Distancia Leveshtein)
#===========================================

df['oracion_corregida']=''
# df = dfo.copy()
for index, row in df.iterrows():
  oracion = row['oracion_final_str']
  lista_palabras = []
  for palabra in oracion:    
    lista_palabras.append(spellsp.correction(palabra))
  # print(lista_palabras)  
  df.at[index,'oracion_corregida'] = lista_palabras
print(df.shape)
df.head(3)

(9659, 8)


,CODIGO_MODULAR,SERIE_EQUIPO_OSEE,obs,flg_cat,flg_bin,oracion_final_str,length,oracion_corregida
1,200220,T0000000002,La niña vive con su abuela Aurelia Costilla Fl...,6,1,"[ni, a, vivir, abuela, aurelia, costilla, flor...",10,"[ni, a, vivir, abuela, aurelia, costilla, flor..."
2,200238,T0000000003,Se asignó la tableta a la estudiante pero aún ...,14,0,"[asignar, tableta, estudiante, entregar, estud...",6,"[asignar, tableta, estudiante, entregar, estud..."
3,200220,T0000000004,La Madre vive en Anexo y no pudo actualizar su...,1,0,"[madre, vivir, anexo, actualizar, dni, motivo,...",7,"[madre, vivir, anexo, actualizar, dni, motivo,..."


In [21]:
# Quitando palabras que no tienen corrección
#===========================================

from hunspell import Hunspell
h = Hunspell('index', hunspell_data_dir='')
h.spell('orgulloupn')

def EliminarStopWordsHunspell(tokens):
    tks=[]
    for token in tokens:
      if len(token)>1:
        res = h.spell(token)
        if res:
          tks.append(token)
    return tks

In [22]:
df['oracion_corregida']= df['oracion_corregida'].swifter.apply(lambda oracion: EliminarStopWordsHunspell(oracion))
df['length_final'] = df.oracion_corregida.apply(lambda oracion: len(oracion))
df = df[df['length_final']>=3]
df['oracion_corregida'] = df['oracion_corregida'].swifter.apply(lambda x: ' '.join([str(item) for item in x]))
print(df.shape)
df.head(5)

Pandas Apply:   0%|          | 0/9659 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7804 [00:00<?, ?it/s]

(7804, 9)


,CODIGO_MODULAR,SERIE_EQUIPO_OSEE,obs,flg_cat,flg_bin,oracion_final_str,length,oracion_corregida,length_final
1,200220,T0000000002,La niña vive con su abuela Aurelia Costilla Fl...,6,1,"[ni, a, vivir, abuela, aurelia, costilla, flor...",10,ni vivir abuela costilla flores padre fallecer,7
2,200238,T0000000003,Se asignó la tableta a la estudiante pero aún ...,14,0,"[asignar, tableta, estudiante, entregar, estud...",6,asignar tableta estudiante entregar estudiante...,6
3,200220,T0000000004,La Madre vive en Anexo y no pudo actualizar su...,1,0,"[madre, vivir, anexo, actualizar, dni, motivo,...",7,madre vivir anexo actualizar motivo pandemia,6
4,200220,T0000000005,La madre vive en Anexo con la Estudiante y no ...,1,0,"[madre, vivir, anexo, estudiante, validar, dni...",7,madre vivir anexo estudiante validar pandemia,6
5,200220,T0000000006,La madre vive en Anexo y no pudo realizar actu...,1,0,"[madre, vivir, anexo, actualización, dni, pand...",6,madre vivir anexo actualización pandemia,5


In [23]:
df=df[['obs','oracion_corregida']]
print(df.shape)
df.head(3)

(7804, 2)


,obs,oracion_corregida
1,La niña vive con su abuela Aurelia Costilla Fl...,ni vivir abuela costilla flores padre fallecer
2,Se asignó la tableta a la estudiante pero aún ...,asignar tableta estudiante entregar estudiante...
3,La Madre vive en Anexo y no pudo actualizar su...,madre vivir anexo actualizar motivo pandemia


# 2. Generate of BoW

In [24]:
tweet = df["oracion_corregida"]
tweet.head()

1       ni vivir abuela costilla flores padre fallecer
2    asignar tableta estudiante entregar estudiante...
3         madre vivir anexo actualizar motivo pandemia
4        madre vivir anexo estudiante validar pandemia
5             madre vivir anexo actualización pandemia
Name: oracion_corregida, dtype: object

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(tweet)

tdm = pd.DataFrame(X.toarray().transpose(), index=vec.get_feature_names())
tdm.columns = tweet.index
tdm.head()

1      2      3      4      5      6      7      9      10     \
abad            0      0      0      0      0      0      0      0      0   
abajo           0      0      0      0      0      0      0      0      0   
abandonado      0      0      0      0      0      0      0      0      0   
abandonar       0      0      0      0      0      0      0      0      0   
abandono        0      0      0      0      0      0      0      0      0   

            11     12     13     14     15     18     19     21     22     \
abad            0      0      0      0      0      0      0      0      0   
abajo           0      0      0      0      0      0      0      0      0   
abandonado      0      0      0      0      0      0      0      0      0   
abandonar       0      0      0      0      0      0      0      0      0   
abandono        0      0      0      0      0      0      0      0      0   

            23     24     25     26     27     28     29     34     35     \
abad            0      0      0      0      0      0      0      0      0   
abajo           0      0      0      0      0      0      0      0      0   
abandonado      0      0      0      0      0      0      0      0      0   
abandonar       0      0      0      0      0      0      0      0      0   
abandono        0      0      0      0      0      0      0      0      0   

            36     38     39     40     41     43     44     46     56     \
abad            0      0      0      0      0      0      0      0      0   
abajo           0      0      0      0      0      0      0      0      0   
abandonado      0      0      0      0      0      0      0      0      0   
abandonar       0      0      0      0      0      0      0      0      0   
abandono        0      0      0      0      0      0      0      0      0   

            58     59     62     63     69     70     71     72     74     \
abad            0      0      0      0      0      0      0      0      0   
abajo           0      0      0      0      0      0      0      0      0   
abandonado      0      0      0      0      0      0      0      0      0   
abandonar       0      0      0      0      0      0      0      0      0   
abandono        0      0      0      0      0      0      0      0      0   

            75     78     79     83     85     86     89     91     117    \
abad            0      0      0      0      0      0      0      0      0   
abajo           0      0      0      0      0      0      0      0      0   
abandonado      0      0      0      0      0      0      0      0      0   
abandonar       0      0      0      0      0      0      0      0      0   
abandono        0      0      0      0      0      0      0      0      0   

            121    125    126    131    137    139    157    159    160    \
abad            0      0      0      0      0      0      0      0      0   
abajo           0      0      0      0      0      0      0      0      0   
abandonado      0      0      0      0      0      0      0      0      0   
abandonar       0      0      0      0      0      0      0      0      0   
abandono        0      0      0      0      0      0      0      0      0   

            161    163    167    169    172    183    189    190    192    \
abad            0      0      0      0      0      0      0      0      0   
abajo           0      0      0      0      0      0      0      0      0   
abandonado      0      0      0      0      0      0      0      0      0   
abandonar       0      0      0      0      0      0      0      0      0   
abandono        0      0      0      0      0      0      0      0      0   

            194    195    204    206    208    210    211    213    224    \
abad            0      0      0      0      0      0      0      0      0   
abajo           0      0      0      0      0      0      0      0      0   
abandonado      0      0      0      0      0      0      0      0      0   


In [26]:
matcor = pd.DataFrame(np.corrcoef(tdm.values, rowvar=False), columns=tdm.columns)
print(matcor.shape)
matcor.head()

(7804, 7804)


1         2         3         4         5         6         7      \
0  1.000000 -0.002187  0.152168  0.152168  0.167121 -0.001785 -0.002306   
1 -0.002187  1.000000 -0.002024  0.287241 -0.001848 -0.001431 -0.001848   
2  0.152168 -0.002024  1.000000  0.665887  0.729728 -0.001652 -0.002134   
3  0.152168  0.287241  0.665887  1.000000  0.729728 -0.001652 -0.002134   
4  0.167121 -0.001848  0.729728  0.729728  1.000000 -0.001508 -0.001948   

      9         10        11        12        13        14        15     \
0 -0.002526 -0.002526  0.187325 -0.002526 -0.001785  0.187325  0.187325   
1  0.721140  0.721140  0.352486  0.142608 -0.001431  0.352486  0.352486   
2 -0.002338 -0.002338  0.407135 -0.002338 -0.001652  0.611657  0.407135   
3  0.164718  0.164718  0.611657 -0.002338 -0.001652  0.611657  0.611657   
4 -0.002134 -0.002134  0.446256 -0.002134 -0.001508  0.446256  0.446256   

      18        19        21        22        23        24        25     \
0 -0.002306 -0.002187 -0.001785 -0.001785 -0.002306 -0.002062 -0.002526   
1  0.314965  0.874781  0.202992  0.202992  0.156559 -0.001652 -0.002024   
2 -0.002134 -0.002024 -0.001652 -0.001652 -0.002134 -0.001908 -0.002338   
3  0.180831  0.287241 -0.001652 -0.001652 -0.002134 -0.001908 -0.002338   
4 -0.001948 -0.001848 -0.001508 -0.001508 -0.001948 -0.001742 -0.002134   

      26        27        28        29        34        35        36     \
0 -0.002526 -0.002526 -0.002574 -0.002526 -0.002062 -0.001785 -0.002526   
1 -0.002024 -0.002024 -0.002063 -0.002024  0.175417  0.202992 -0.002024   
2 -0.002338 -0.002338 -0.002383 -0.002338 -0.001908 -0.001652 -0.002338   
3 -0.002338 -0.002338 -0.002383 -0.002338 -0.001908 -0.001652 -0.002338   
4 -0.002134 -0.002134 -0.002175 -0.002134 -0.001742 -0.001508 -0.002134   

      38        39        40        41        43        44        46     \
0 -0.002729 -0.001785 -0.002526 -0.002306  0.199402 -0.001785 -0.002526   
1 -0.002187 -0.001431  0.287241 -0.001848  0.186874  0.202992 -0.002024   
2  0.152168  0.234463  0.164718 -0.002134  0.106391 -0.001652 -0.002338   
3  0.152168  0.470578  0.331774 -0.002134  0.106391 -0.001652 -0.002338   
4  0.167121  0.257093  0.180831 -0.001948  0.117083 -0.001508 -0.002134   

      56        58        59        62        63        69        70     \
0 -0.002062  0.216858 -0.002526 -0.002918 -0.002306 -0.001785 -0.001785   
1 -0.001652 -0.001431 -0.002024 -0.002338  0.156559  0.202992  0.202992   
2 -0.001908 -0.001652 -0.002338 -0.002701 -0.002134 -0.001652 -0.001652   
3 -0.001908 -0.001652 -0.002338 -0.002701 -0.002134 -0.001652 -0.001652   
4 -0.001742 -0.001508 -0.002134 -0.002465 -0.001948 -0.001508 -0.001508   

      71        72        74        75        78        79        83     \
0 -0.002062 -0.002062 -0.001785 -0.002729 -0.002062 -0.001785 -0.003096   
1  0.175417 -0.001652  0.202992  0.399603 -0.001652  0.202992 -0.002481   
2 -0.001908 -0.001908 -0.001652 -0.002526 -0.001908 -0.001652 -0.002865   
3 -0.001908 -0.001908 -0.001652  0.152168 -0.001908 -0.001652 -0.002865   
4 -0.001742 -0.001742 -0.001508 -0.002306 -0.001742 -0.001508 -0.002615   

      85        86        89        91        117       121       125    \
0 -0.003264 -0.002729 -0.002306 -0.002062 -0.001785 -0.002306 -0.002306   
1  0.333743 -0.002187  0.314965  0.529556 -0.001431 -0.001848 -0.001848   
2 -0.003021  0.152168 -0.002134 -0.001908 -0.001652 -0.002134 -0.002134   
3  0.126481  0.152168  0.180831  0.202613 -0.001652 -0.002134 -0.002134   
4 -0.002757  0.167121 -0.001948 -0.001742 -0.001508 -0.001948 -0.001948   

      126       131       137       139       157       159       160    \
0 -0.001785 -0.001785 -0.002062 -0.001785 -0.002306 -0.001785 -0.002729   
1 -0.001431 -0.001431  0.529556  0.202992 -0.001848 -0.001431  0.131743   
2  0.234463 -0.001652 -0.001908 -0.001652 -0.002134 -0.001652 -0.002526   
3 -0.001652 -0.001652  0.202613 -0.001652 -0.002134 -0.001652 -0.002526   
4 -0.001508 -0.001508 -

In [27]:
#Transformamos en la forma de un grafo
#=====================================

cordf = pd.DataFrame()
cordf = pd.DataFrame(columns = ['inicio', 'fin', 'peso'])
for i in matcor.index:
    for j in matcor.index:
        if i<j:
            try:
                w=matcor.loc[i,j]
                cordf = cordf.append({'inicio' : i, 'fin' : j, 'peso' : w}, ignore_index = True)
            except Exception:
                pass

KeyboardInterrupt: 

In [ ]:
#Filtramos para las correlaciones más altas
#==========================================

cordf = cordf[cordf['peso']>.4]
cordf.sort_values('peso', ascending=False).head(10)